##1.Configuración del ambiente

In [ ]:
global df_bitcoin, precio_actual,tendencia, media_bitcoin, algoritmo_decision,df_bitcoin_limpio

In [ ]:
!python -V
print('-----')

Python 3.10.12
-----


In [ ]:
!pip show Pandas | grep 'Name\|Version'
print('-----')

Name: pandas
Version: 1.5.3
-----


In [ ]:
!pip show Numpy | grep 'Name\|Version'
print('-----')

Name: numpy
Version: 1.23.5
-----


In [ ]:
!pip show matplotlib | grep 'Name\|Version'
print('-----')

Name: matplotlib
Version: 3.7.1
-----


In [ ]:
import pandas as pd  # Importar pandas
import numpy as np   # Importar numpy
import matplotlib.pyplot as plt  # Importar matplotlib
import yfinance as yf  # Para obtener datos históricos
import datetime  # para usar en parte 2.1

import requests
import bs4
from bs4 import BeautifulSoup


## 2.Obtención de datos

**2.1 PRECIOS HISTÓRICOS DEL BITCOIN**

In [ ]:
def importar_base_bitcoin():
  global df_bitcoin, precio_actual,tendencia, media_bitcoin, algoritmo_decision,df_bitcoin_limpio
  today = datetime.date.today()
  last_week = today-datetime.timedelta(days=7)
  BTC_USD = yf.download('BTC-USD', start =last_week, end = today,interval="5m")
  df_bitcoin = pd.DataFrame(BTC_USD)

**2.2 INDICADORES DE TENDENCIAS**


In [ ]:
def extraer_tendencias():
  global df_bitcoin, precio_actual,tendencia, media_bitcoin, algoritmo_decision,df_bitcoin_limpio
  tend=[]

  response = requests.get('https://coinmarketcap.com/')
  soup = BeautifulSoup(response.text,"html.parser")

  datos = soup.find_all('td',style='text-align:end')

  for p in datos:
    tend.append(p.text.strip())

  Bitcoin = tend[0]
  Precio1h = tend[1]

  Bitcoin=Bitcoin.replace('$',"").replace(",","")
  Precio1h = Precio1h.replace('%',"")

  precio_actual = float(Bitcoin)
  Precio1h = float(Precio1h)


  t = str(datos)
  index1 = t.find("iwhBxy")
  index2 = t.find("gUnzUB")

  if index1==211:
    tendencia ="Alta"
  if index2==211:
    tendencia="Baja"
  print("------------------------------------")
  print('Bitcoin price : ',precio_actual)
  print('Tendencia : ',tendencia)
  print("------------------------------------")

## 3.Limpieza de datos

In [ ]:
def limpieza_datos():
  global df_bitcoin, precio_actual,tendencia, media_bitcoin, algoritmo_decision,df_bitcoin_limpio
  data_duplicado=[]
  suma=0
  IIQ=0

  #copia del dataframa

  df_bitcoin_limpio = df_bitcoin.copy()  #Copia de base de datos, para trabajar en ella

  duplicado = df_bitcoin_limpio.index.duplicated()   # Buscar duplicados en el índice

  for i in duplicado:                     # Se buscan filas del índice que estén duplicada
    if i == True:
      data_duplicado.append(suma)
    suma=suma+1
  df_bitcoin_limpio.drop(data_duplicado,inplace=True) # Se eliminan posibles duplicados en el índice ( Datetime )

  #------------------------------------------------------------#

  #Búsqueda de nulos en columna CLOSE

  nulos_en_CLOSE = df_bitcoin_limpio['Close'].isnull()

  df_bitcoin_limpio = df_bitcoin_limpio.dropna(subset =["Close"])  # Se eliminan filas con nulos en columna CLOSE

  #----------------------------------------------------------------#

  # Eliminación de filas con 0 en columna Volume

  df_bitcoin_limpio= df_bitcoin_limpio.drop(df_bitcoin_limpio[df_bitcoin_limpio["Volume"] == 0].index)

  #-----------------------------------------------------------------#

  plt.subplot(121)
  plt.title("Gráfico con todos los datos")

  # Gráfico de boxplot para la columna Close

  df_bitcoin_limpio.boxplot(['Close'])

  Q1 = np.quantile(df_bitcoin_limpio['Close'],0.25)
  Q3 = np.quantile(df_bitcoin_limpio['Close'],0.75)

  IIQ = Q3-Q1

  print('Límite inferior', Q1-1.5*IIQ)
  print('Límite superior', Q3+1.5*IIQ)

  print('Valor de Q1:',Q1)
  print('Valor de Q3:',Q3)

  seleccion = ((df_bitcoin_limpio["Close"]>=Q1) & (df_bitcoin_limpio["Close"]<= Q3))

  df_bitcoin_limpio = df_bitcoin_limpio[seleccion]

  plt.subplot(122)
  plt.title("Gráfico con datos entre Q1 y Q3")

  # Gráfico de boxplot para la columna Close

  df_bitcoin_limpio.boxplot(['Close'])



  #--------------------------------------------------------------------#
  # Cálculo de la media
  media_bitcoin = df_bitcoin_limpio['Close'].mean()
  print("--------------------------------------------------------------------------------")
  print("La media de la columna Close con el nuevo rango Q1,Q3 es :", media_bitcoin)
  print("--------------------------------------------------------------------------------")



## 4. Tomar decisiones

In [ ]:
def tomar_decisiones():
  global df_bitcoin, precio_actual,tendencia, media_bitcoin, algoritmo_decision,df_bitcoin_limpio


  if (precio_actual >= media_bitcoin and tendencia =='Baja'):
    algoritmo_decision = "Vender"
  elif (precio_actual <= media_bitcoin and tendencia =="Alta"):
    algoritmo_decision ="Comprar"
  else:
    algoritmo_decision="Esperar"


  print("El algoritmo de decisión recomienda :", algoritmo_decision)


##5. Visualización

In [ ]:
def visualizacion():
  global df_bitcoin, precio_actual,tendencia, media_bitcoin, algoritmo_decision,df_bitcoin_limpio
  df_bitcoin['Promedio']=media_bitcoin

  fig = plt.figure(figsize=(16,5))
  plt.title("Análisis de valor BITCOIN para 7 días")
  x=df_bitcoin.index
  y=df_bitcoin.Close
  plt.plot(x,y)

  x=df_bitcoin.index
  y=df_bitcoin.Promedio
  plt.plot(x,y,color = "red")


  plt.annotate(algoritmo_decision, xy=(df_bitcoin.index[900],df_bitcoin.Close[1200]),xytext=(df_bitcoin.index[900],df_bitcoin.Close[1200]),fontsize=20,color="green")
  plt.show()



##6. Automatización

In [ ]:
from IPython.display import clear_output
import time


while(True):
  clear_output()
  importar_base_bitcoin()
  extraer_tendencias()
  limpieza_datos()
  tomar_decisiones()
  visualizacion()
  time.sleep(300)